In this file, I prepare the data for the height of the left and right heels.
This involves
- loading the data from the c3d files
- selecting the relevant portion of the timeseries (because the recording started before the drummer began playing and ended after they finished)
- normalizing the data by subtracting the minimum height
- saving the data as two csv files (one for right, one for left heels), where 
    - rows are participants, 
    - columns are songs, and 
    - each cell contains the time series for the given heel height.


In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import pyc3dserver as c3d
%matplotlib inline

import utils

Some useful functions

In [5]:
# load data and parameters
participant_folders = utils.get_participants_data_folders()
participant_numbers = utils.load_participant_numbers()
song_genres = utils.load_song_genres()
song_names = utils.load_song_names()
z_dim = utils.zdim() # up and down

In [6]:
# initialize the dataframes where we'll store the data
timeseries_l = pd.DataFrame(columns=song_names, index=participant_numbers)
timeseries_l[song_names] = timeseries_l[song_names].astype(object) # because each cell will contain an array
timeseries_r = timeseries_l.copy(deep=True)

# Get the COM object of C3Dserver
itf = c3d.c3dserver()

for participant_folder in participant_folders:
    # list all c3d files in directory
    c3d_files = sorted(list(participant_folder.glob('*.c3d')))
    
    for file in c3d_files:
        print(file)
        # get the song and participant identifiers
        song_name = utils.path_to_song_name(file)
        song_number = utils.path_to_song_number(file)
        participant_number = utils.path_to_participant_number(file)
        
        # get the data for the given file
        ret = c3d.open_c3d(itf, file)
        markers = c3d.get_dict_markers(itf)

        # get the raw data for the given marker in the z-dimension
        start_frame, end_frame = utils.get_start_end_frames(song_name, participant_number)
        raw_l = markers['DATA']['POS']['LHLO'][start_frame : end_frame, z_dim]
        raw_r = markers['DATA']['POS']['RHLO'][start_frame : end_frame, z_dim]

        # normalize the data by subtracting the minimum height
        normalized_l = raw_l - np.min(raw_l)
        normalized_r = raw_r - np.min(raw_r)
        
        # store data
        timeseries_l.loc[participant_number, song_name] = [list(normalized_l)]
        timeseries_r.loc[participant_number, song_name] = [list(normalized_r)]

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-01-Bloodline-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-02-BackInBlack-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-03-Juju-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-04-SoWhat-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-05-TheOcean-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-06-GoIntoTheWater-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P2-Filled-in\220127-P2-01-TheOcean-Filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P2-Filled-in\220127-P2-02-Juju-Filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_proj

In [7]:
l_filename = Path().resolve().parent / "data" / "data-prep_mocap-of-heels_left.csv"
timeseries_l.to_csv(l_filename, index=True)

r_filename = Path().resolve().parent / "data" / "data-prep_mocap-of-heels_right.csv"
timeseries_r.to_csv(r_filename, index=True)
